# Prueba una

In [14]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
from google.cloud import storage
import os


In [9]:
sites_atributos = pd.read_parquet("../Data/dfgy_attributes.parquet")

In [10]:
# Se observan los atributos.
sites_atributos["attributes"].unique()

array(['RestaurantsDelivery', 'OutdoorSeating',
       'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking',
       'RestaurantsPriceRange2', 'RestaurantsTakeOut',
       'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters',
       'WheelchairAccessible', 'GoodForKids', 'RestaurantsAttire',
       'RestaurantsReservations', 'Ambience', 'CoatCheck', 'DogsAllowed',
       'RestaurantsTableService', 'RestaurantsGoodForGroups', 'HasTV',
       'HappyHour', 'DriveThru', 'NoiseLevel', 'GoodForMeal',
       'BusinessAcceptsBitcoin', 'Smoking', 'Music', 'GoodForDancing',
       'BestNights', 'BYOB', 'Corkage', 'BYOBCorkage', 'AcceptsInsurance',
       'RestaurantsCounterService', 'Open24Hours', 'AgesAllowed',
       'DietaryRestrictions', 'HairSpecializesIn', 'Good for kids',
       'Restroom', 'Bar onsite', 'Wi-Fi', 'Gender-neutral restroom',
       'Public restroom', 'High chairs', 'Restaurant', 'Golf course',
       'Mechanic', 'Air conditioning', 'Bar on site', 'Toilets',
       'Gen

In [11]:
# Se seleccionan los atributos que van a ser unificados o corregidos a través de un diccionario de mapeo.
reemplazo_atributos = {
    "Accessible entrance":['Wheelchair accessible entrance','Wheelchair-accessible entrance',"WheelchairAccessible"],
    "Accessible elevator":["Wheelchair accessible elevator","Wheelchair-accessible lift"],
    "Accessible seating":['Wheelchair accessible seating','Wheelchair-accessible seating'],
    "Accessible restroom":['Wheelchair accessible restroom','Wheelchair-accessible toilet'],
    "Accessible parking":['Wheelchair accessible parking lot','Wheelchair-accessible car park'],
    "Wifi":['WiFi','Wi-Fi'],
    "Delivery&TakeOut":["RestaurantsDelivery",'RestaurantsTakeOut'],
    "Outdoor Seating":['OutdoorSeating'],
    "Accepts Cards":['BusinessAcceptsCreditCards',"BusinessAcceptsBitcoin"],
    "Parking":["BusinessParking",'BikeParking'],
    "Appointment Only":['ByAppointmentOnly'],
    "Reservations":['RestaurantsReservations'],
    "HappyHour":['HappyHour','BestNights'],
    "BYOB":['BYOB', 'Corkage', 'BYOBCorkage'],
    'Good for kids':['Good for kids','GoodForKids'],
    "Sells Alcohol":['Alcohol','Bar onsite','Bar on site'],
    "Restroom":['Restroom','Gender-neutral restroom','Public restroom','Toilets','Gender-neutral toilets','Public toilet'],
    'Baggage storage':['Baggage storage','CoatCheck']

}

# Se seleccionan los atributos que no proporcionan información útil para el proceso de análisis.
eliminar_atributos = ['Caters','RestaurantsPriceRange2','Ambience','RestaurantsTableService','NoiseLevel',
                      'Music','AcceptsInsurance','AgesAllowed','HairSpecializesIn','High chairs', 'Restaurant','Golf course',
                      'Mechanic','Swimming pool', 'All-inclusive', 'Stadium seating']

In [12]:
# Se invierte el diccionario de mapeo para usarlo en el método "replace".
reverse_mapping = {v: k for k, values in reemplazo_atributos.items() for v in values}

# Se reeemplazan los valores en la columna 'atributos' usando el diccionario invertido
sites_atributos['attributes'] = sites_atributos['attributes'].replace(reverse_mapping)

# Se filtra y eliminan las filas que contienen atributos en eliminar_atributos
sites_atributos = sites_atributos[~sites_atributos['attributes'].str.contains('|'.join(eliminar_atributos))]

In [13]:
# Se observan las categorías limpias.
sites_atributos["attributes"].unique()

array(['Delivery&TakeOut', 'Outdoor Seating', 'Accepts Cards', 'Parking',
       'Appointment Only', 'Wifi', 'Sells Alcohol', 'Accessible entrance',
       'Good for kids', 'Reservations', 'Baggage storage', 'DogsAllowed',
       'HasTV', 'HappyHour', 'DriveThru', 'GoodForMeal', 'Smoking',
       'GoodForDancing', 'BYOB', 'Open24Hours', 'DietaryRestrictions',
       'Restroom', 'Air conditioning', 'Accessible elevator',
       'Accessible seating', 'Accessible restroom', 'Accessible parking',
       'Assisted listening devices'], dtype=object)

In [15]:
# Subida del dataframe a Google Cloud 

# Configura tus credenciales de Google Cloud
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/credentials.json'

# Función que sube el archivo a Google Cloud Storage
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Sube un archivo a un bucket de Google Cloud Storage."""
    # Inicializa el cliente de GCS
    storage_client = storage.Client()
    # Obtén el bucket
    bucket = storage_client.bucket(bucket_name)
    # Crea un blob en el bucket
    blob = bucket.blob(destination_blob_name)
    # Sube el archivo al blob
    blob.upload_from_filename(source_file_name)
    print(f"Archivo {source_file_name} subido a {destination_blob_name}.")

# Función que guarda el dataframe en un archivo temporal de parquet y lo sube
def dataframe_to_parquet_and_upload(df, bucket_name, destination_blob_name):
    # Guarda el DataFrame como archivo Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'temp.parquet')

    # Sube el archivo Parquet a Google Cloud Storage
    upload_to_gcs(bucket_name, 'temp.parquet', destination_blob_name)

    # Elimina el archivo temporal
    os.remove('temp.parquet')

In [17]:
# Se sube el dataframe de atributos

# Nombre del bucket de GCS y del archivo destino
bucket_name = 'scripts-python-proyecto-henry'
destination_blob_name = 'dfgy_attributes.parquet'

dataframe_to_parquet_and_upload(sites_atributos,bucket_name,destination_blob_name)

Archivo temp.parquet subido a dfgy_attributes.parquet.


# Prueba Dos

In [3]:
import pandas as pd
import os

In [1]:
# Lista para almacenar los dataframes de todos los estados
dataframes = []

# Función para procesar cada estado y ruta
def procesar_archivos_json(state, ruta_directorio, chunksize=10000):
    for filename in os.listdir(ruta_directorio):
        if filename.endswith('.json'):
            try:
                filepath = os.path.join(ruta_directorio, filename)
                with pd.read_json(filepath, lines=True, chunksize=chunksize) as reader:
                    for chunk in reader:
                        chunk['state'] = state
                        dataframes.append(chunk)
            except Exception as e:
                print(f"Error al procesar el archivo {filename}: {e}")

In [4]:
# Diccionario de estados y rutas
estados_ruta = {
    "AL": "../Data/Google/reviews-estados/review-Alabama/review-Alabama",
    'AK':'../Data/Google/reviews-estados/review-Alaska/review-Alaska',
    'AZ':'../Data/Google/reviews-estados/review-Arizona/review-Arizona',
    'AR':'../Data/Google/reviews-estados/review-Arkansas/review-Arkansas',
    'CA':'../Data/Google/reviews-estados/review-California/review-California',
    'CO':'../Data/Google/reviews-estados/review-Colorado/review-Colorado',
    'CT':'../Data/Google/reviews-estados/review-Connecticut/review-Connecticut',
    'DE':"../Data/Google/reviews-estados/review-Delaware/review-Delaware",
    'District_of_Columbia':"../Data/Google/reviews-estados/review-District_of_Columbia",
    'FL':"../Data/Google/reviews-estados/review-Florida/review-Florida",
    'GA':"../Data/Google/reviews-estados/review-Georgia/review-Georgia",
    'HI':"../Data/Google/reviews-estados/review-Hawaii/review-Hawaii",
    'ID':"../Data/Google/reviews-estados/review-Idaho/review-Idaho",
    'IL':"../Data/Google/reviews-estados/review-Illinois/review-Illinois",
    'IN':"../Data/Google/reviews-estados/review-Indiana/review-Indiana",
    'IA':"../Data/Google/reviews-estados/review-Iowa/review-Iowa",
    'KS':"../Data/Google/reviews-estados/review-Kansas/review-Kansas",
    'KY':"../Data/Google/reviews-estados/review-Kentucky/review-Kentucky",
    'LA':"../Data/Google/reviews-estados/review-Louisiana/review-Louisiana",
    'ME':"../Data/Google/reviews-estados/review-Maine/review-Maine",
    'MD':"../Data/Google/reviews-estados/review-Maryland/review-Maryland",
    'MA':"../Data/Google/reviews-estados/review-Massachusetts/review-Massachusetts",
    'MI':"../Data/Google/reviews-estados/review-Michigan/review-Michigan",
    'MN':"../Data/Google/reviews-estados/review-Minnesota/review-Minnesota",
    'MS':"../Data/Google/reviews-estados/review-Mississippi/review-Mississippi",
    'MO':"../Data/Google/reviews-estados/review-Missouri/review-Missouri",
    'MT':"../Data/Google/reviews-estados/review-Montana/review-Montana",
    'NE':"../Data/Google/reviews-estados/review-Nebraska/review-Nebraska",
    'NV':"../Data/Google/reviews-estados/review-Nevada/review-Nevada",
    'NH':"../Data/Google/reviews-estados/review-New_Hampshire/review-New_Hampshire",
    'NJ':"../Data/Google/reviews-estados/review-New_Jersey/review-New_Jersey",
    'NM':"../Data/Google/reviews-estados/review-New_Mexico/review-New_Mexico",
    'NY':"../Data/Google/reviews-estados/review-New_York/review-New_York",
    'NC':"../Data/Google/reviews-estados/review-North_Carolina/review-North_Carolina",
    'ND':"../Data/Google/reviews-estados/review-North_Dakota/review-North_Dakota",
    'OH':"../Data/Google/reviews-estados/review-Ohio/review-Ohio",
    'OK':"../Data/Google/reviews-estados/review-Oklahoma/review-Oklahoma",
    'OR':"../Data/Google/reviews-estados/review-Oregon/review-Oregon",
    'PA':"../Data/Google/reviews-estados/review-Pennsylvania/review-Pennsylvania",
    'RI':"../Data/Google/reviews-estados/review-Rhode_Island/review-Rhode_Island",
    'SC':"../Data/Google/reviews-estados/review-South_Carolina/review-South_Carolina",
    'SD':"../Data/Google/reviews-estados/review-South_Dakota/review-South_Dakota",
    'TN':"../Data/Google/reviews-estados/review-Tennessee/review-Tennessee",
    'TX':"../Data/Google/reviews-estados/review-Texas/review-Texas",
    'UT':"../Data/Google/reviews-estados/review-Utah/review-Utah",
    'VT':"../Data/Google/reviews-estados/review-Vermont/review-Vermont",
    'VA':"../Data/Google/reviews-estados/review-Virginia/review-Virginia",
    'WA':"../Data/Google/reviews-estados/review-Washington/review-Washington",
    'WV':"../Data/Google/reviews-estados/review-West_Virginia/review-West_Virginia",
    'WI':"../Data/Google/reviews-estados/review-Wisconsin/review-Wisconsin",
    'WY':"../Data/Google/reviews-estados/review-Wyoming/review-Wyoming"
}


# Iterar sobre el diccionario y procesar los archivos
for state, ruta_directorio in estados_ruta.items():
    procesar_archivos_json(state, ruta_directorio)

# Concatenar todos los dataframes en uno solo
dfg_reviews = pd.concat(dataframes, ignore_index=True)

# Mostrar el dataframe final
print(dfg_reviews)

# tablas BigQuery

In [1]:
from google.cloud import bigquery
import pandas as pd

# Crear un cliente de BigQuery
client = bigquery.Client()

In [2]:
datasets = list(client.list_datasets())
if datasets:
    print("Datasets en el proyecto {}:".format(client.project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} no tiene conjuntos de datos.".format(client.project))

Datasets en el proyecto proyecto-nuevo-423502:
	Data_Henry_Final


In [3]:
# Atributos
query = """
    SELECT *
    FROM `proyecto-nuevo-423502.Data_Henry_Final.atributos`
"""

query_job = client.query(query)  # Ejecutar la consulta
results = query_job.result()
atributos = results.to_dataframe()

# Categorias
query = """
    SELECT *
    FROM `proyecto-nuevo-423502.Data_Henry_Final.categorias`
"""

query_job = client.query(query)  # Ejecutar la consulta
results = query_job.result()
categorias = results.to_dataframe()

# Restaurantes
query = """
    SELECT *
    FROM `proyecto-nuevo-423502.Data_Henry_Final.restaurantes`
"""

query_job = client.query(query)  # Ejecutar la consulta
results = query_job.result()
restaurantes = results.to_dataframe()

# Usuarios
query = """
    SELECT *
    FROM `proyecto-nuevo-423502.Data_Henry_Final.usuarios`
"""

query_job = client.query(query)  # Ejecutar la consulta
results = query_job.result()
usuarios = results.to_dataframe()


# Reviews
query = """
    SELECT *
    FROM `proyecto-nuevo-423502.Data_Henry_Final.reviews`
"""

query_job = client.query(query)  # Ejecutar la consulta
results = query_job.result()
reviews = results.to_dataframe()


c:\Users\dmon2\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
atributos.head()

# Dataset reviews final con texto

In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq
import pyarrow as pa
import random
from google.cloud import storage

In [2]:
# Se crea una función para subir los dataframes preprocesados a Google Cloud Storage

# Configura tus credenciales de Google Cloud
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/credentials.json'

# Función que sube el archivo a Google Cloud Storage
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Sube un archivo a un bucket de Google Cloud Storage."""
    # Inicializa el cliente de GCS
    storage_client = storage.Client()
    # Obtén el bucket
    bucket = storage_client.bucket(bucket_name)
    # Crea un blob en el bucket
    blob = bucket.blob(destination_blob_name)
    # Sube el archivo al blob
    blob.upload_from_filename(source_file_name)
    print(f"Archivo {source_file_name} subido a {destination_blob_name}.")

# Función que guarda el dataframe en un archivo temporal de parquet y lo sube
def dataframe_to_parquet_and_upload(df, bucket_name, destination_blob_name):
    # Guarda el DataFrame como archivo Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'temp.parquet')

    # Sube el archivo Parquet a Google Cloud Storage
    upload_to_gcs(bucket_name, 'temp.parquet', destination_blob_name)

    # Elimina el archivo temporal
    os.remove('temp.parquet')

In [3]:
# Nombre del bucket de GCS
bucket_name = 'archivos-preprocesados-henry'

In [4]:
# Definir el porcentaje de muestra que deseas
sample_fraction = 0.10

# Leer el archivo parquet
parquet_file = "../Data/dfgy_reviews_text.parquet"

# Leer metadatos del archivo parquet
parquet_metadata = pq.ParquetFile(parquet_file).metadata
num_rows = parquet_metadata.num_rows

# Calcular el número de filas a tomar en la muestra
sample_size = int(num_rows * sample_fraction)

# Leer el dataset completo
df = pd.read_parquet(parquet_file)

# Tomar una muestra aleatoria del 30% de los datos
sample_df = df.sample(frac=sample_fraction, random_state=42)

KeyboardInterrupt: 

In [ ]:
reviews_text.info()